In [1]:
#load "Extensions.fs"
open Extensions

type Point = int * int
type Line = Point * Point

let testInput = ["498,4 -> 498,6 -> 496,6"; "503,4 -> 502,4 -> 502,9 -> 494,9"]
let fullInput = File.ReadAllLines "inputs/14/input.txt"

let parseBlockLines input =
  input
  |> Seq.collect (
    String.split " -> "
    >> List.map (String.split "," >> List.map int >> List.toPair)
    >> List.windowed 2
    >> List.map (List.sortBy snd >> List.sortBy fst >> List.toPair))
  |> Seq.toList

parseBlockLines testInput

index Item1 Item2 0 Item1 Item2 498 4 Item1 Item2 498 6 1 Item1 Item2 496 6 Item1 Item2 498 6 2 Item1 Item2 502 4 Item1 Item2 503 4 3 Item1 Item2 502 4 Item1 Item2 502 9 4 Item1 Item2 494 9 Item1 Item2 502 9

In [2]:
let containsPoint ((x,y): Point) = function
  | (x1,y1),(x2,y2) when x1 = x2 -> x = x1 && y1 <= y && y <= y2
  | (x1,y1),(x2,y2) when y1 = y2 -> y = y1 && x1 <= x && x <= x2
  | _ -> failwith "Only supports horizontal or vertical lines"

let bounds (points: Point seq) =
  Seq.allPairs (Seq.map Seq.map [fst;snd]) [Seq.min; Seq.max]
  |> Seq.map ((<||) (>>) >> (|>) points)
  |> Seq.toList
  |> fun [a;b;c;d] -> {|MinX = a; MaxX = b; MinY = c; MaxY = d|}

type CaveSpace = Rock | Air | Sand | SandSource with
  override this.ToString() =
    match this with
    | Rock -> "#"
    | Sand -> "o"
    | SandSource -> "+"
    | Air -> "."

[<StructuredFormatDisplay("")>]
type GridScan = {
    Rocks: Set<Line>
    Sand: Set<Point>
    SandSource: Point
  } with
  member this.Item with get p = 
    match p with
    | p when Set.exists (containsPoint p) this.Rocks -> Rock
    | p when Set.contains p this.Sand -> Sand
    | p when p = this.SandSource -> SandSource
    | _ -> Air

  member this.Bounds =
    this.Rocks
    |> Seq.collect List.ofPair
    |> Seq.append this.Sand
    |> Seq.append (Seq.singleton this.SandSource)
    |> bounds

  override this.ToString() = 
    seq {
      for y = this.Bounds.MinY to this.Bounds.MaxY do
        for x = this.Bounds.MinX to this.Bounds.MaxX do
          yield this[x,y].ToString()
        yield "\n"
    } |> String.Concat  

  static member empty = { Rocks = Set.empty; Sand = Set.empty; SandSource = (500,0) }
  static member parse input = { GridScan.empty with Rocks = Set.ofSeq (parseBlockLines input) }

GridScan.parse testInput |> printf "%O" 

......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
........#.
........#.
#########.


In [3]:
type GridScan with
  member this.dropSand = function
    | (_,y) when y > (this.Rocks |> Seq.collect List.ofPair |> Seq.map snd |> Seq.max) -> None
    | (x,y) -> 
      match Seq.allPairs [x;x-1;x+1] [y+1] |> Seq.tryFind (fun p -> this[p] = Air) with
      | Some next -> this.dropSand next
      | None -> Some (x,y)
  member this.addSand pt = {this with Sand = Set.add pt this.Sand}
  static member generateSand (this:GridScan) = this.dropSand this.SandSource |> Option.map this.addSand

(GridScan.parse testInput)
|> GridScan.generateSand
|> Option.iter (printfn "%O")

......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
........#.
......o.#.
#########.



In [4]:
let generateAllSand = Seq.unfold (GridScan.generateSand >> Option.map (fun x -> x,x))

let testGrid = GridScan.parse testInput
let testResults = 
  testGrid :: (testGrid |> generateAllSand |> Seq.toList)

testResults[24] |> printfn "%O"

......+...
..........
......o...
.....ooo..
....#ooo##
...o#ooo#.
..###ooo#.
....oooo#.
.o.ooooo#.
#########.



In [5]:
let fullGrid = GridScan.parse fullInput
fullGrid |> generateAllSand |> Seq.length

843

In [6]:
let withFloor (grid: GridScan) =
  let floor = grid.Bounds.MaxY + 2
  { grid with Rocks = grid.Rocks |> Set.add ((Int32.MinValue, floor),(Int32.MaxValue, floor)) }

let fill (grid: GridScan) = 
  let rec flood filled = function
    | p when Set.contains p filled || grid[p] = Rock -> filled
    | x,y as p when List.contains grid[x,y] [SandSource; Air] -> 
      Seq.allPairs [x-1;x;x+1] [y+1]
      |> Seq.fold (fun s (x,y) -> flood s (x,y)) (Set.add p filled)
  flood Set.empty grid.SandSource

testGrid
|> withFloor
|> fill
|> Set.count

93

In [7]:
let fullSand = fullGrid |> withFloor |> fill
fullSand |> Set.count

27625

In [12]:
#r "nuget: Plotly.NET"
#r "nuget: Plotly.NET.Interactive"
open Plotly.NET
open Plotly.NET.LayoutObjects


Installed Packages Plotly.NET, 3.0.1 Plotly.NET.Interactive, 3.0.2

In [17]:
let rocks = fullGrid.Rocks |> Seq.map (fun ((x1,y1),(x2,y2)) -> Shape.init(StyleParam.ShapeType.Rectangle, x1, x2, y1, y2))

fullSand
|> Chart.Point
|> Chart.withShapes(rocks)
|> Chart.withYAxis(LinearAxis.init(AutoRange = StyleParam.AutoRange.Reversed))

<!-- Plotly chart will be drawn inside this DIV -->